In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 필요 시 설치(가상환경의 경우 터미널에 설치)
!pip install transformers==4.24.0

In [ ]:
# 필요 시 설치(가상환경의 경우 터미널에 설치)
!pip install datasets==2.6.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 12.8 MB/s 
     |████████████████████████████████| 115 kB 67.9 MB/s 
     |████████████████████████████████| 212 kB 59.3 MB/s 
     |████████████████████████████████| 95 kB 5.8 MB/s 
     |████████████████████████████████| 127 kB 88.0 MB/s 
     |████████████████████████████████| 115 kB 72.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


# 모듈 import 및 전역 변수 설정

In [ ]:
import json
import os

import torch
import torch.nn as nn
from tqdm import trange
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from datasets import load_metric
from sklearn.metrics import f1_score
import pandas as pd
import copy

PADDING_TOKEN = 1
S_OPEN_TOKEN = 0
S_CLOSE_TOKEN = 2

do_eval=True

# 학습된 모델이 저장될 경로 설정
category_extraction_model_path = ''

# 모델을 이어서 학습할 시 불러올 파일 경로 설정
test_category_extraction_model_path = ''

# 학습할 json 데이터 설정
train_data_path = '/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/nikluge-sa-2022-test.jsonl'
# 학습 시 evaluation할 json 데이터 설정
dev_data_path = '/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/nikluge-sa-2022-test.jsonl'
# 학습 후 저장된 모델(pt file)로 predict파일을 만들 json 데이터 설정
test_data_path = '/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/nikluge-sa-2022-test.jsonl'

# 최대 단어 수 설정
max_len = 256
# 배치 사이즈 설정
batch_size = 8
# 학습시 사용할 베이스 모델 설정
base_model = 'kykim/electra-kor-base'
# 러닝 레이트 설정
learning_rate = 3e-6
# eps 설정
eps = 1e-8
# 학습 할 에폭 수 설정
num_train_epochs = 
classifier_hidden_size = 768
classifier_dropout_prob = 0.1

# 개체#속성 쌍 설정(label25)
entity_property_pair = [
    '제품 전체#일반', '제품 전체#가격', '제품 전체#디자인', '제품 전체#품질', '제품 전체#편의성', '제품 전체#인지도', '제품 전체#다양성',
    '본품#일반', '본품#디자인', '본품#품질', '본품#편의성', '본품#다양성', '본품#가격', '본품#인지도',
    '패키지/구성품#일반', '패키지/구성품#디자인', '패키지/구성품#품질', '패키지/구성품#편의성', '패키지/구성품#가격', '패키지/구성품#다양성',
    '브랜드#일반', '브랜드#가격', '브랜드#품질', '브랜드#인지도', '브랜드#디자인',
                    ]

# 문장과 개체#속성 쌍의 관계를 True, False 로 표시
tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

# 그래픽 카드 사용하는 변수
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 불러온 tokenizer 에 special token 을 추가
special_tokens_dict = {
    'additional_special_tokens': ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
}

SyntaxError: ignored

json 및 jsonl 파일 read, write 함수

In [ ]:
# json 파일 읽어서 list에 저장
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j


# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

# 모델 정의
kykim_electra 모델을 기반으로 한 classification 모델 이용

In [ ]:
# 아래 ElectraBaseClassifier의 classifier로 사용될 class
class SimpleClassifier(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        self.dense = nn.Linear(classifier_hidden_size, classifier_hidden_size)
        self.dropout = nn.Dropout(classifier_dropout_prob)
        self.output = nn.Linear(classifier_hidden_size, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

# 불러올 base model 기반 classification class 생성
class ElectraBaseClassifier(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ElectraBaseClassifier, self).__init__()

        self.num_label = num_label
        self.ky_electra = AutoModel.from_pretrained(base_model)
        self.ky_electra.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.ky_electra(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits


# 데이터 파싱 및 tokenization 함수 정의


In [ ]:
# 데이터를 tokenizer화
def tokenize_and_align_labels(tokenizer, form, annotations, max_len):

    # entity_property 데이터를 넣을 빈 딕셔너리 생성
    entity_property_data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'label': []
    }

    # 미리 만들어 둔 entity_property_pair 리스트 내에 '속성#개체' 쌍에 대하여 각각 tokenizer 후 label 달아줌
    for pair in entity_property_pair:
        isPairInOpinion = False
        if pd.isna(form):
            break
        tokenized_data = tokenizer(form, pair, padding='max_length', max_length=max_len, truncation=True)
        for annotation in annotations:
            entity_property = annotation[0]

            # entity_property 는 True 일 때 tonkenizer 후 label 달아줌
            if entity_property == pair:
                entity_property_data_dict['input_ids'].append(tokenized_data['input_ids'])
                entity_property_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
                entity_property_data_dict['label'].append(tf_name_to_id['True'])

                isPairInOpinion = True
                break

        # entity_property 는 False 일 때 작업하지 않음
        if isPairInOpinion is False:
            entity_property_data_dict['input_ids'].append(tokenized_data['input_ids'])
            entity_property_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
            entity_property_data_dict['label'].append(tf_name_to_id['False'])

    return entity_property_data_dict

# 데이터 셋 형성 함수
def get_dataset(raw_data, tokenizer, max_len):
    # tokenizer 된 entity_property 데이터가 들어갈 빈 리스트 생성 
    input_ids_list = []
    attention_mask_list = []
    token_labels_list = []

    # 문장별로 tokenize_and_align_labels 를 통해 tokenizer 후 각각의 리스트에 입력
    for utterance in raw_data:
        entity_property_data_dict = tokenize_and_align_labels(tokenizer, utterance['sentence_form'], utterance['annotation'], max_len)
        input_ids_list.extend(entity_property_data_dict['input_ids'])
        attention_mask_list.extend(entity_property_data_dict['attention_mask'])
        token_labels_list.extend(entity_property_data_dict['label'])

    return TensorDataset(torch.tensor(input_ids_list), torch.tensor(attention_mask_list), torch.tensor(token_labels_list))

# 모델 학습

In [ ]:
# f1 score 를 통해 성능을 확인하는 함수
def evaluation(y_true, y_pred, label_len):
    count_list = [0]*label_len
    hit_list = [0]*label_len
    for i in range(len(y_true)):
        count_list[y_true[i]] += 1
        if y_true[i] == y_pred[i]:
            hit_list[y_true[i]] += 1
    acc_list = []

    for i in range(label_len):
        acc_list.append(hit_list[i]/count_list[i])

    print(count_list)
    print(hit_list)
    print(acc_list)
    print('accuracy: ', (sum(hit_list) / sum(count_list)))
    print('macro_accuracy: ', sum(acc_list) / 3)
    # print(y_true)

    y_true = list(map(int, y_true))
    y_pred = list(map(int, y_pred))

    print('f1_score: ', f1_score(y_true, y_pred, average=None))
    print('f1_score_micro: ', f1_score(y_true, y_pred, average='micro'))
    print('f1_score_macro: ', f1_score(y_true, y_pred, average='macro'))

# 모델을 학습시키는 함수
def train_sentiment_analysis():

    print('train_sentiment_analysis')
    print('category_extraction model would be saved at ', category_extraction_model_path)

    print('loading train data')
    # train 데이터가 jsonl일 경우 위 코드 그대로 json일 경우는 아래 주석처리된 코드로 바꿔서 사용
    train_data = jsonlload(train_data_path)
    # train_data = jsonload(train_data_path)

    # dev 데이터가 jsonl일 경우 위 코드 그대로 json일 경우는 아래 주석처리된 코드로 바꿔서 사용
    dev_data = jsonlload(dev_data_path)
    # dev_data = jsonload(dev_data_path)

    print('tokenizing train data')
    # pretrained base model tokenizer 불러옴
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    # tokenizer 에 추가할 special token 설정
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
    print('We have added', num_added_toks, 'tokens')
    # get_dataset 함수 를 통해 entity_property, polarity train 데이터셋을 생성
    entity_property_train_data = get_dataset(train_data, tokenizer, max_len)
    # get_dataset 함수 를 통해 entity_property, polarity dev 데이터셋을 생성
    entity_property_dev_data = get_dataset(dev_data, tokenizer, max_len)
    # DataLoader 를 통해 위에서 만든 entity_property_train_data 를 배치 사이즈 수 단위로 나누고, 데이터를 셔플
    entity_property_train_dataloader = DataLoader(entity_property_train_data, shuffle=True,
                                  batch_size=batch_size)
    # DataLoader 를 통해 위에서 만든 entity_property_dev_data 를 배치 사이즈 수 단위로 나누고, 데이터를 셔플
    entity_property_dev_dataloader = DataLoader(entity_property_dev_data, shuffle=True,
                                batch_size=batch_size)

    # 사용할 모델 불러오기
    print('loading model')
    entity_property_model = ELECTRABaseClassifier(len(tf_id_to_name), len(tokenizer))
    # 학습된 pt값에 이어서 학습할 시 아래 주석 제거
    # entity_property_model.load_state_dict(torch.load(test_category_extraction_model_path, map_location=device))
    # 그래픽 카드 사용
    entity_property_model.to(device)

    print('end loading')

    # entity_property_optimizer 매개변수 분리
    FULL_FINETUNING = True
    if FULL_FINETUNING:
        entity_property_param_optimizer = list(entity_property_model.named_parameters())
        no_decay = ['bias', 'gamma', 'beta']
        entity_property_optimizer_grouped_parameters = [
            {'params': [p for n, p in entity_property_param_optimizer if not any(nd in n for nd in no_decay)],
             'weight_decay_rate': 0.01},
            {'params': [p for n, p in entity_property_param_optimizer if any(nd in n for nd in no_decay)],
             'weight_decay_rate': 0.0}
        ]
    else:
        entity_property_param_optimizer = list(entity_property_model.classifier.named_parameters())
        entity_property_optimizer_grouped_parameters = [{"params": [p for n, p in entity_property_param_optimizer]}]

    # entity_property_optimizer_setting
    entity_property_optimizer = AdamW(
        entity_property_optimizer_grouped_parameters,
        lr=learning_rate,
        eps=eps
    )
    epochs = num_train_epochs
    max_grad_norm = 1.0
    total_steps = epochs * len(entity_property_train_dataloader)

    # entity_property_scheduler_setting
    entity_property_scheduler = get_linear_schedule_with_warmup(
        entity_property_optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # 모델이 위에서 설정 한 에폭수 만큼 돌아감
    epoch_step = 0

    for _ in trange(epochs, desc="Epoch"):
        entity_property_model.train()
        epoch_step += 1

        # entity_property train
        entity_property_total_loss = 0

        for step, batch in enumerate(entity_property_train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            entity_property_model.zero_grad()

            loss, _ = entity_property_model(b_input_ids, b_input_mask, b_labels)

            loss.backward()

            entity_property_total_loss += loss.item()

            torch.nn.utils.clip_grad_norm_(parameters=entity_property_model.parameters(), max_norm=max_grad_norm)
            entity_property_optimizer.step()
            entity_property_scheduler.step()

        # loss 값 계산
        avg_train_loss = entity_property_total_loss / len(entity_property_train_dataloader)
        print("Entity_Property_Epoch: ", epoch_step)
        print("Average train loss: {}".format(avg_train_loss))

        # 미리 설정한 경로에 모델 저장
        model_saved_path = category_extraction_model_path + '/saved_model_epoch_' + str(epoch_step+21) + '.pt'
        torch.save(entity_property_model.state_dict(), model_saved_path)

        # do_eval의 설정에 따라 evaluation 수행
        if do_eval:
            entity_property_model.eval()

            pred_list = []
            label_list = []

            for batch in entity_property_dev_dataloader:
                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels = batch

                with torch.no_grad():
                    loss, logits = entity_property_model(b_input_ids, b_input_mask, b_labels)

                predictions = torch.argmax(logits, dim=-1)
                pred_list.extend(predictions)
                label_list.extend(b_labels)

            evaluation(label_list, pred_list, len(tf_id_to_name))

    print("training is done")


In [ ]:
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
train_sentiment_analysis()

# 모델 평가

학습된 모델을 바탕으로 국어원 데이터 형태를 만드는 방법 예시

In [ ]:
def predict_from_korean_form(tokenizer, ce_model, pc_model, data):

    ce_model.to(device)
    ce_model.eval()
    for sentence in data:
        form = sentence['sentence_form']
        sentence['annotation'] = []
        if type(form) != str:
            print("form type is arong: ", form)
            continue
        for pair in entity_property_pair:
            

            tokenized_data = tokenizer(form, pair, padding='max_length', max_length=256, truncation=True)

            input_ids = torch.tensor([tokenized_data['input_ids']]).to(device)
            attention_mask = torch.tensor([tokenized_data['attention_mask']]).to(device)
            with torch.no_grad():
                _, ce_logits = ce_model(input_ids, attention_mask)

            ce_predictions = torch.argmax(ce_logits, dim = -1)

            ce_result = tf_id_to_name[ce_predictions[0]]

            if ce_result == 'True':
                with torch.no_grad():
                    _, pc_logits = pc_model(input_ids, attention_mask)

                pc_predictions = torch.argmax(pc_logits, dim=-1)
                pc_result = polarity_id_to_name[pc_predictions[0]]

                sentence['annotation'].append([pair, pc_result])


    return data


F1 score 계산 - 추출 성능 및 전체 성능에 대한 F1 score 따로 계산

In [ ]:
def evaluation_f1(true_data, pred_data):

    true_data_list = true_data
    pred_data_list = pred_data

    ce_eval = {
        'TP': 0,
        'FP': 0,
        'FN': 0,
        'TN': 0
    }

    pipeline_eval = {
        'TP': 0,
        'FP': 0,
        'FN': 0,
        'TN': 0
    }

    for i in range(len(true_data_list)):

        # TP, FN checking
        is_ce_found = False
        is_pipeline_found = False
        for y_ano  in true_data_list[i]['annotation']:
            y_category = y_ano[0]
            y_polarity = y_ano[2]

            for p_ano in pred_data_list[i]['annotation']:
                p_category = p_ano[0]
                p_polarity = p_ano[1]

                if y_category == p_category:
                    is_ce_found = True
                    if y_polarity == p_polarity:
                        is_pipeline_found = True

                    break

            if is_ce_found is True:
                ce_eval['TP'] += 1
            else:
                ce_eval['FN'] += 1

            if is_pipeline_found is True:
                pipeline_eval['TP'] += 1
            else:
                pipeline_eval['FN'] += 1

            is_ce_found = False
            is_pipeline_found = False

        # FP checking
        for p_ano in pred_data_list[i]['annotation']:
            p_category = p_ano[0]
            p_polarity = p_ano[1]

            for y_ano  in true_data_list[i]['annotation']:
                y_category = y_ano[0]
                y_polarity = y_ano[2]

                if y_category == p_category:
                    is_ce_found = True
                    if y_polarity == p_polarity:
                        is_pipeline_found = True

                    break

            if is_ce_found is False:
                ce_eval['FP'] += 1

            if is_pipeline_found is False:
                pipeline_eval['FP'] += 1

    ce_precision = ce_eval['TP']/(ce_eval['TP']+ce_eval['FP'])
    ce_recall = ce_eval['TP']/(ce_eval['TP']+ce_eval['FN'])

    ce_result = {
        'Precision': ce_precision,
        'Recall': ce_recall,
        'F1': 2*ce_recall*ce_precision/(ce_recall+ce_precision)
    }

    pipeline_precision = pipeline_eval['TP']/(pipeline_eval['TP']+pipeline_eval['FP'])
    pipeline_recall = pipeline_eval['TP']/(pipeline_eval['TP']+pipeline_eval['FN'])

    pipeline_result = {
        'Precision': pipeline_precision,
        'Recall': pipeline_recall,
        'F1': 2*pipeline_recall*pipeline_precision/(pipeline_recall+pipeline_precision)
    }

    return {
        'category extraction result': ce_result,
        'entire pipeline result': pipeline_result
    }


테스트 데이터에 대한 평가

In [ ]:
# test 데이터에 라벨링 하기 위해 entity_model 학습 한 pt 값 설정
test_entity_extraction_model_path = ''
# test 데이터에 라벨링 하기 위해 property_model 학습 한 pt 값 설정
test_property_extraction_model_path = ''
# 학습 후 저장된 pt파일로 predict파일을 만들 json 데이터 설정
test_data_path = ''

In [ ]:
def test_sentiment_analysis():

    tokenizer = AutoTokenizer.from_pretrained(base_model)
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
    test_data = jsonlload(test_data_path)
    
    model = ELECTRABaseClassifier(len(tf_id_to_name), len(tokenizer))
    model.load_state_dict(torch.load(test_category_extraction_model_path, map_location=device))
    model.to(device)
    model.eval()
            
    polarity_model = ELECTRABaseClassifier(len(polarity_id_to_name), len(tokenizer))
    polarity_model.load_state_dict(torch.load(test_polarity_classification_model_path, map_location=device))
    polarity_model.to(device)
    polarity_model.eval()

    pred_data = predict_from_korean_form(tokenizer, model, polarity_model, copy.deepcopy(test_data))

    jsondump(pred_data, '/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/kykim_train_dev_backtranslation/pred_data.json')
    # pred_data = jsonload('/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/kykim_train_dev_backtranslation/pred_data.json')

    # print('F1 result: ', evaluation_f1(test_data, pred_data))

In [ ]:
test_sentiment_analysis()